<a href="https://colab.research.google.com/github/GiovanniReguzzoni/modelliNLP/blob/main/bertALBERTOhate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import csv

In [ ]:
!pip install datasets transformers huggingface_hub

     |████████████████████████████████| 346 kB 5.0 MB/s 
     |████████████████████████████████| 4.2 MB 54.1 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 140 kB 42.2 MB/s 
     |████████████████████████████████| 212 kB 56.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.3 MB/s 
     |████████████████████████████████| 127 kB 47.4 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 271 kB 45.3 MB/s 
     |████████████████████████████████| 144 kB 18.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
E

In [ ]:
!nvidia-smi

Mon May 23 09:37:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    32W / 250W |  14345MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from datasets import Dataset


In [ ]:
dftestl = pd.read_feather('/content/dftestlHSfea')

In [ ]:
dftrain = pd.read_feather('/content/dftrainHSfea')

In [ ]:
dftest1 = pd.read_feather('/content/dftestfea')

In [ ]:
dftest1.columns

Index(['id', 'text'], dtype='object')

In [ ]:
dftrain = dftrain.rename(columns={'text ': 'text'})
dftestl = dftestl.rename(columns={'text ': 'text'})

In [ ]:
dftrain = dftrain.rename(columns={'hs': 'label'})
dftestl = dftestl.rename(columns={'hs': 'label'})

In [ ]:
dftest1 = dftest1.drop('id', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
dftest1

,text
0,Che fine spero che faccia il killer nigeriano ...
1,Così i profughi ci svuotano i negozi a Pordenone
2,Così umiliano gli italiani e coccolano i cland...
3,"Danno soldi ai clandestini, ma ai disabili inv..."
4,"Emergenza, Minniti annulla il viaggio e torna ..."
...,...
495,Il pm fa sbarcare i migranti Il capo leghista ...
496,"Pisa, il poster di Salvini con i migranti fatt..."
497,Sea Watch e lo sbarco del migrante con una sol...
498,"Decreto Sicurezza Bis, multe più salate per ch..."


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# AlBERTO tokenizer
# m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", model_max_length=512)

In [ ]:

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [ ]:

train = Dataset.from_pandas(dftrain)
test = Dataset.from_pandas(dftestl)

In [ ]:
test1 = Dataset.from_pandas(dftest1)

In [ ]:
train.features


{'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None)}

In [ ]:
tokentrain = train.map(preprocess_function, batched=True)
tokentest = test.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#test senza label
tokentest1 = test1.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokentrain

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6837
})

In [ ]:
tokentest

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500
})

In [ ]:

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")


In [ ]:
print(encoded_input)


{'input_ids': [2, 839, 2941, 14308, 28907, 133, 24, 291, 44793, 572, 63947, 1, 795, 20093, 1052, 5437, 60220, 1015, 24014, 367, 75919, 1, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", num_labels=2)

Downloading:   0%|          | 0.00/706M [00:00<?, ?B/s]

Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:



from transformers import TrainingArguments, Trainer

repo_name = "AlbertoBertHateSpeech"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokentrain ,
    eval_dataset=tokentest ,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/GioReg/AlbertoBertHateSpeech into local empty directory.


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6837
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1712


Step,Training Loss
500,0.456800
1000,0.286300
1500,0.166900


Saving model checkpoint to AlbertoBertHateSpeech/checkpoint-428
Configuration saved in AlbertoBertHateSpeech/checkpoint-428/config.json
Model weights saved in AlbertoBertHateSpeech/checkpoint-428/pytorch_model.bin
tokenizer config file saved in AlbertoBertHateSpeech/checkpoint-428/tokenizer_config.json
Special tokens file saved in AlbertoBertHateSpeech/checkpoint-428/special_tokens_map.json
tokenizer config file saved in AlbertoBertHateSpeech/tokenizer_config.json
Special tokens file saved in AlbertoBertHateSpeech/special_tokens_map.json
Saving model checkpoint to AlbertoBertHateSpeech/checkpoint-856
Configuration saved in AlbertoBertHateSpeech/checkpoint-856/config.json
Model weights saved in AlbertoBertHateSpeech/checkpoint-856/pytorch_model.bin
tokenizer config file saved in AlbertoBertHateSpeech/checkpoint-856/tokenizer_config.json
Special tokens file saved in AlbertoBertHateSpeech/checkpoint-856/special_tokens_map.json
Saving model checkpoint to AlbertoBertHateSpeech/checkpoint-12

TrainOutput(global_step=1712, training_loss=0.2787549651671793, metrics={'train_runtime': 1511.5953, 'train_samples_per_second': 18.092, 'train_steps_per_second': 1.133, 'total_flos': 7195561141985280.0, 'train_loss': 0.2787549651671793, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16


{'epoch': 4.0,
 'eval_accuracy': 0.764,
 'eval_f1': 0.572463768115942,
 'eval_loss': 1.2055922746658325,
 'eval_runtime': 9.8076,
 'eval_samples_per_second': 50.981,
 'eval_steps_per_second': 3.263}

# Performing Inference
https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python

In [ ]:
trainer.push_to_hub()



Saving model checkpoint to AlbertoBertHateSpeech
Configuration saved in AlbertoBertHateSpeech/config.json
Model weights saved in AlbertoBertHateSpeech/pytorch_model.bin
tokenizer config file saved in AlbertoBertHateSpeech/tokenizer_config.json
Special tokens file saved in AlbertoBertHateSpeech/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/703M [00:00<?, ?B/s]

Upload file runs/May23_08-04-04_dfd1d6139cba/events.out.tfevents.1653293075.dfd1d6139cba.72.0:  75%|#######5  …

Upload file runs/May23_08-04-04_dfd1d6139cba/events.out.tfevents.1653295093.dfd1d6139cba.72.2: 100%|##########…

KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline


In [ ]:
sentiment_model = pipeline(task= 'sentiment-analysis', 
                      model= "/content/AlbertoBertHateSpeech",
                      tokenizer = "/content/AlbertoBertHateSpeech")

loading configuration file /content/AlbertoBertHateSpeech/config.json
Model config BertConfig {
  "_name_or_path": "/content/AlbertoBertHateSpeech",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 128000
}

loading 

In [ ]:
#sentiment_model = pipeline(model="GioReg/ita1")

In [ ]:
sentiment_model(["Malaria? E' solo l'inizio.... La tremenda"])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_0', 'score': 0.7508717775344849}]

In [ ]:
list_text = dftest1.text.tolist()

In [ ]:
listat ={}

In [ ]:
listat= sentiment_model(list_text)

In [ ]:
listat

[{'label': 'LABEL_0', 'score': 0.998860239982605},
 {'label': 'LABEL_1', 'score': 0.9921230673789978},
 {'label': 'LABEL_1', 'score': 0.9978099465370178},
 {'label': 'LABEL_0', 'score': 0.9991105198860168},
 {'label': 'LABEL_0', 'score': 0.997377872467041},
 {'label': 'LABEL_0', 'score': 0.9948180317878723},
 {'label': 'LABEL_1', 'score': 0.8508855104446411},
 {'label': 'LABEL_1', 'score': 0.9961177110671997},
 {'label': 'LABEL_1', 'score': 0.9949066042900085},
 {'label': 'LABEL_0', 'score': 0.9988927245140076},
 {'label': 'LABEL_0', 'score': 0.9924030303955078},
 {'label': 'LABEL_1', 'score': 0.9966732263565063},
 {'label': 'LABEL_1', 'score': 0.5186395645141602},
 {'label': 'LABEL_1', 'score': 0.9979403614997864},
 {'label': 'LABEL_1', 'score': 0.9966126084327698},
 {'label': 'LABEL_1', 'score': 0.8337366580963135},
 {'label': 'LABEL_0', 'score': 0.9431002736091614},
 {'label': 'LABEL_1', 'score': 0.694391667842865},
 {'label': 'LABEL_0', 'score': 0.6182816624641418},
 {'label': 'LAB

In [ ]:
print(listat)

[{'label': 'LABEL_0', 'score': 0.998860239982605}, {'label': 'LABEL_1', 'score': 0.9921230673789978}, {'label': 'LABEL_1', 'score': 0.9978099465370178}, {'label': 'LABEL_0', 'score': 0.9991105198860168}, {'label': 'LABEL_0', 'score': 0.997377872467041}, {'label': 'LABEL_0', 'score': 0.9948180317878723}, {'label': 'LABEL_1', 'score': 0.8508855104446411}, {'label': 'LABEL_1', 'score': 0.9961177110671997}, {'label': 'LABEL_1', 'score': 0.9949066042900085}, {'label': 'LABEL_0', 'score': 0.9988927245140076}, {'label': 'LABEL_0', 'score': 0.9924030303955078}, {'label': 'LABEL_1', 'score': 0.9966732263565063}, {'label': 'LABEL_1', 'score': 0.5186395645141602}, {'label': 'LABEL_1', 'score': 0.9979403614997864}, {'label': 'LABEL_1', 'score': 0.9966126084327698}, {'label': 'LABEL_1', 'score': 0.8337366580963135}, {'label': 'LABEL_0', 'score': 0.9431002736091614}, {'label': 'LABEL_1', 'score': 0.694391667842865}, {'label': 'LABEL_0', 'score': 0.6182816624641418}, {'label': 'LABEL_1', 'score': 0.9

In [ ]:
df = pd.DataFrame.from_dict(listat)

In [ ]:
df.head()

,label,score
0,LABEL_0,0.998860
1,LABEL_1,0.992123
2,LABEL_1,0.997810
3,LABEL_0,0.999111
4,LABEL_0,0.997378


In [ ]:
df['label'] = df['label'].str.replace('LABEL_', '')

In [ ]:
df.head(6)

,label,score
0,0,0.998860
1,1,0.992123
2,1,0.997810
3,0,0.999111
4,0,0.997378
5,0,0.994818


In [ ]:
df = df.rename(columns={'label': 'predizione'})

In [ ]:
dftestl.head()

,text,label
0,Che fine spero che faccia il killer nigeriano ...,1
1,Così i profughi ci svuotano i negozi a Pordenone,1
2,Così umiliano gli italiani e coccolano i cland...,1
3,"Danno soldi ai clandestini, ma ai disabili inv...",1
4,"Emergenza, Minniti annulla il viaggio e torna ...",1


In [ ]:
result=pd.concat([dftestl, df], axis=1)

In [ ]:
result.head(20)

,text,label,predizione,score
0,Che fine spero che faccia il killer nigeriano ...,1,0,0.998860
1,Così i profughi ci svuotano i negozi a Pordenone,1,1,0.992123
2,Così umiliano gli italiani e coccolano i cland...,1,1,0.997810
3,"Danno soldi ai clandestini, ma ai disabili inv...",1,0,0.999111
4,"Emergenza, Minniti annulla il viaggio e torna ...",1,0,0.997378
5,"Gli immigrati sono matti, parola di scienziati...",1,0,0.994818
6,Ho chiesto ai profughi di lavorare ma preferis...,1,1,0.850886
7,Immigrati bomba sociale Subito via 600mila irr...,1,1,0.996118
8,"Immigrati, così la sinistra ci costringe a ten...",1,1,0.994907
9,"Immigrati, venite tutti qui. L'ultima ""perla"" ...",1,0,0.998893


In [ ]:
result.dtypes

text           object
label           int64
predizione     object
score         float64
dtype: object

In [ ]:
result["predizione"] = pd.to_numeric(df["predizione"])

In [ ]:
result.dtypes

text           object
label           int64
predizione      int64
score         float64
dtype: object

In [ ]:
conditions = [result['label'] == result['predizione'], result['label'] != result['predizione']]


choices = ['corretta','sbagliata']

result['risultato'] = np.select(conditions, choices, default='Errore')

result.head(20)

,text,label,predizione,score,risultato
0,Che fine spero che faccia il killer nigeriano ...,1,0,0.998860,sbagliata
1,Così i profughi ci svuotano i negozi a Pordenone,1,1,0.992123,corretta
2,Così umiliano gli italiani e coccolano i cland...,1,1,0.997810,corretta
3,"Danno soldi ai clandestini, ma ai disabili inv...",1,0,0.999111,sbagliata
4,"Emergenza, Minniti annulla il viaggio e torna ...",1,0,0.997378,sbagliata
5,"Gli immigrati sono matti, parola di scienziati...",1,0,0.994818,sbagliata
6,Ho chiesto ai profughi di lavorare ma preferis...,1,1,0.850886,corretta
7,Immigrati bomba sociale Subito via 600mila irr...,1,1,0.996118,corretta
8,"Immigrati, così la sinistra ci costringe a ten...",1,1,0.994907,corretta
9,"Immigrati, venite tutti qui. L'ultima ""perla"" ...",1,0,0.998893,sbagliata


In [ ]:
result['risultato'].value_counts()

corretta     382
sbagliata    118
Name: risultato, dtype: int64

In [ ]:
x=result['risultato'].value_counts()

In [ ]:
x[0] / (x[0]+x[1])

0.764

In [ ]:
#accuracy
from sklearn import metrics
metrics.accuracy_score(result['label'], result['predizione'])

0.764

In [ ]:
#precision
from sklearn.metrics import precision_score
precision_score(result['label'], result['predizione'],  average='macro')

0.7898635477582846

In [ ]:
#calcolo recall
from sklearn.metrics import recall_score

recall_score(result['label'], result['predizione'],  average='macro')

0.693153674292939

In [ ]:
 from sklearn.metrics import f1_score
#f1
f1_score(result['label'], result['predizione'], average='micro')

In [ ]:
 from sklearn.metrics import confusion_matrix

In [ ]:
array=confusion_matrix(result['label'], result['predizione'])


In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ax= plt.subplot()


df_cm = pd.DataFrame(array, index = [i for i in "01"],
                  columns = [i for i in "01"])

sn.heatmap(df_cm, cmap="YlGnBu", fmt='g', annot=True)

In [ ]:
pip install disarray

In [ ]:
df = pd.DataFrame(array, index= ['0','1'], columns=['0','1'])

In [ ]:
import disarray

In [ ]:
df.da.export_metrics()